# 匯入套件
## 標準套件: numpy, pandas, matplotlib




In [ ]:
%matplotlib inline
# 強制讓圖顯示在這個網頁，不再開新視窗

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcdefaults()

### 0. 讀入你打造好的 vector dataset

In [24]:
!pip -q install gdown

In [25]:
GDRIVE_PUBLIC_URL = "https://drive.google.com/file/d/16axitUWzN0zPeL-O24pVP86lT_EdeXOK/view?usp=drive_link"

In [26]:
!gdown --fuzzy -O faiss_db.zip "{GDRIVE_PUBLIC_URL}"

Downloading...
From: https://drive.google.com/uc?id=16axitUWzN0zPeL-O24pVP86lT_EdeXOK
To: /content/faiss_db.zip
100% 86.6k/86.6k [00:00<00:00, 11.6MB/s]


In [27]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
replace faiss_db/index.faiss? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


### 1. 安裝並引入必要套件

In [28]:
!pip install -U langchain langchain-community faiss-cpu transformers sentence-transformers huggingface_hub
!pip -q install "aisuite[all]"

In [29]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationalRetrievalChain

In [30]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [31]:
import os
from google.colab import userdata

In [32]:
hf_token = userdata.get('HuggingFace')

In [33]:
from huggingface_hub import login
login(token=hf_token)

In [34]:
class EmbeddingGemmaEmbeddings(HuggingFaceEmbeddings):
    def __init__(self, **kwargs):
        super().__init__(
            model_name="google/embeddinggemma-300m",
            encode_kwargs={"normalize_embeddings": True},
            **kwargs
        )

    def embed_documents(self, texts):
        # 你也可以把 "none" 改成真實標題（檔名/章節名），效果會更穩
        texts = [f"title: none | text: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        # 官方檢索建議前綴
        return super().embed_query(f"task: search result | query: {text}")

### 3. 載入 `faiss_db`

In [35]:
embedding_model = EmbeddingGemmaEmbeddings()
vectorstore = FAISS.load_local(
    "faiss_db",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

In [36]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

### 4. 設定好我們要的 LLM

> 使用 Groq 服務

In [37]:
import aisuite as ai

In [38]:
api_key = userdata.get('Groq')

In [39]:
os.environ['GROQ_API_KEY']=api_key

In [40]:
model = "groq:openai/gpt-oss-120b"

In [41]:
client = ai.Client()

### 5. `prompt` 設計

In [42]:
system_prompt = """
你是一位樂於助人的 Maker 學長，專精於 Arduino 和 ESP32 相關的電子實作。

你的任務是協助正在學習的使用者（例如學弟妹）看懂 WS2812B (或 NeoPixel) 的技術規格書 (datasheet)。

你的回答風格應該：
1. 親切且口語化：用「學長」的語氣，而不是冰冷的機器人。
2. 易於理解：主動用白話文（生活化的例子）去解釋 datasheet 上的技術術語。
3. 附帶建議：在回答規格時，如果可能，主動提供相關的「實作建議」（例如關於供電、訊號線電阻、或推薦使用的函式庫）。
4. 台灣用語：請使用台灣習慣的中文回應。
"""

prompt_template = """
[任務開始]
請你扮演 Maker 學長的角色。

根據以下的「WS2812B 規格書資料」：
---
{retrieved_chunks}
---

回答使用者的問題：「{question}」

回答要求：
1.  **[首要任務]** 你的回答必須「嚴格基於」上述提供的「WS2812B 規格書資料」。
2.  **[角色扮演]** 用易於理解的白話文解釋資料中的答案，並視情況提供具體的實作建議。
3.  **[後備指令]** 如果在上述資料中「明確找不到」能回答「{question}」的資訊，
請**不要猜測**，並必須回覆：「學弟/妹，關於『{question}』這個問題，我查了手邊的 WS2812B 規格書，
但裡面沒有提到這部分的具體資訊耶。這類資訊（例如耗電量計算、散熱建議）通常會需要參考你的具體應用電路設計，
或是查看你使用的函式庫（像是 Adafruit NeoPixel 或 FastLED）的說明文件喔。」

[回答範例]
(如果問時序 T0H)
「喔！你問到重點了，這個 T0H 就是 0-code 的高電位時間。根據 datasheet (在 {retrieved_chunks} 裡找到的)，
它的標準時間是 0.4us (微秒)。你在用 Arduino 或 ESP32 寫驅動程式時要特別注意，這個時間如果抓不準，LED 顏色就會亂跳喔！」

(如果問散熱，且資料沒有)
「學弟/妹，關於『散熱片』這個問題，我查了手邊的 WS2812B 規格書，但裡面沒有提到這部分的具體資訊耶。...（後略）」

[任務結束]
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [43]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history
    # 取回相關資料
    # 【已除錯】：將 .get_relevant_documents 替換為 .invoke，以符合 LangChain Runnable 規範
    docs = retriever.invoke(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    # 用 AI Suite 呼叫語言模型
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

### 7. 用 Gradio 打造 Web App

In [45]:
with gr.Blocks(fill_height=True) as demo:
    gr.Markdown("# Maker 學長")
    chatbot = gr.Chatbot(scale=1)
    msg = gr.Textbox(placeholder="請輸入關於 WS2812 的相關問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

/tmp/ipython-input-549549510.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(scale=1)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://37261fff3be4723c79.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://37261fff3be4723c79.gradio.live
